In [ ]:
#This tool maps an image of a human face to a 128 dimensional vector space where images of
# the same person are near to each other and images from different people are far apart.
# Therefore, you can perform face recognition by mapping faces to the 128D space and then
#checking if their Euclidean distance is small enough.

In [ ]:
%pip install gdown
import gdown

# URL for the folder
url = "https://drive.google.com/drive/folders/1ZKMUq6U6pRw7zcP6AkGxoQVX3LnMnGXC?usp=share_link"
gdown.download_folder(url, quiet=True)

In [ ]:
!pip install tqdm
!pip install dlib

In [ ]:
# 23032 images for each feature

import os
import cv2
import numpy as np
import shutil
import zipfile
import dlib
from tqdm import tqdm
from google.colab.patches import cv2_imshow

# Disable CUDA explicitly in dlib
dlib.DLIB_USE_CUDA = False  # Force CPU usage

In [ ]:
face_rec_model_path = "/content/splitted_dataset/dlib_face_recognition_resnet_model_v1.dat"
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

def main():
  folder_path = "/content/splitted_dataset"
  for file_name in os.listdir(folder_path):

    # zipped files in the folder biometric_systems_dataset
    if file_name.endswith(".zip"):

      # full path to the zip file
      file_path = os.path.join(folder_path, file_name)

      # create folder in the same path with the name of the zip file
      extract_folder = os.path.join(folder_path, file_name.replace(".zip", ""))
      os.makedirs(extract_folder, exist_ok=True)

      # open and extract the file
      with zipfile.ZipFile(file_path, mode="r") as zip_ref:
        zip_ref.extractall(extract_folder)

  feature_folders = ["eyes", "nose", "mouth"]
  sets = ["train", "test"]
  for feature in feature_folders:
    feature_folder_path = os.path.join(folder_path, feature)
    os.makedirs(feature, exist_ok=True)

    #remove the extracted zip files
    if feature_folder_path.endswith(".zip"):
      os.remove(feature_folder_path)
    for s in sets:

      # output folder for the embeddings for each feature
      os.makedirs(os.path.join(feature, s), exist_ok=True)
      set_folder_path = os.path.join(feature_folder_path, s)
      for file_name in tqdm(os.listdir(set_folder_path), desc="Processing Images"):
        if file_name.endswith(".jpg"):
          image_path = os.path.join(set_folder_path, file_name)
          image = cv2.imread(image_path)
          image = cv2.resize(image, (150,150))
          face_embedding = facerec.compute_face_descriptor(image)
          np.save(os.path.join(feature, s, file_name.replace(".jpg", ".npy")), np.array(face_embedding))
    shutil.make_archive(feature, "zip", feature)

if __name__ == "__main__":
    main()
